Anexo Q

# Desplegando modelo Red Neuronal en IBM Watson

#### Elaborado por: Ricardo Niño de Rivera Barrón

#### Ingeniería Biónica

#### Trabajo Terminal II

En esta libreta interactiva realizada en IBM Watson se describe el proceso para desplegar un modelo Red Neuronal en la nube utilizando las herramientas que proporcionar gratuitamente IBM.

## Configurando el ambiente

Primeramente es necesario crear una instancia gratuita en Watson para utilizar el servicio específico de Machine Learning.

### Conectando a Watson Machine Learning

Para comunicarnos con la instancia creada necesitamos dos datos, "Api Key" y la ubicación geográfica de la instancia.


In [1]:
api_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
location = 'us-south'

In [2]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Importando la biblioteca para manipular recursos en IBM Watson

In [3]:
#!pip install -U ibm-watson-machine-learning

In [4]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Trabajando con espacios

Posteriormente se debe un "espacio" en el servicio de almacenamiento de objetos de IBM Cloud.

Para realizar esta tarea es posible realizarla en esta libreta o con directamente en la plataforma.

In [7]:
space_id = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

Enlistando todos los espacios existentes en la instancia creada.

In [8]:
client.spaces.list(limit=10)

------------------------------------  ----------------  ------------------------
ID                                    NAME              CREATED
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  EfficientNet_TT2  2020-12-03T14:58:40.571Z
------------------------------------  ----------------  ------------------------


Ahora establecemos el espacio creado como espacio por default para desplegar el modelo propuesto.

In [9]:
client.set.default_space(space_id)

'SUCCESS'

In [10]:
client.deployments.get_job_details()

{'resources': []}

## Descargando modelo y datos necesarios

Instalamos la biblioteca necesaria para comunicación con Mega

In [12]:
!pip install mega.py

     |████████████████████████████████| 13.7 MB 10.2 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 8.9 MB/s  eta 0:00:01
  Created wheel for pathlib: filename=pathlib-1.0.1-py3-none-any.whl size=14346 sha256=f618004721bc029a3fecd0f56565d9684b87be6854309958307c2b59a836e4ca
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/6e/96/b8/10037fe231e23970bac58361d7c93571ab983a7bbc55e68550
Successfully built pathlib


In [13]:
# De la biblioteca mega importamos el método Mega
from mega import Mega

In [14]:
# Instanciando Mega en el objeto mega
mega = Mega()

Inicamos sesión en MEGA con cuenta temporal y anónima

In [15]:
# Log in en la cuenta de MEGA con cuenta temporal anónima
m = mega.login()

In [16]:
# Descargando el archivo X_train.npy
m.download_url('https://mega.nz/file/oAZT1IYJ#HpIvnnR50IH6N2F9wfdPgNTgLPpAwhEBpTl3_yAxLFc')

PosixPath('X_train.npy')

In [17]:
# Mostrando archivos en el directorio de trabajo del entorno
!ls

X_train.npy


In [18]:
# Descargando el archivo X_test.npy
m.download_url('https://mega.nz/file/tMIzDQbK#MLFeDRu5D4dq2lRqvPcxIZkuTtVKH2azBRrhSzndqME')

PosixPath('X_test.npy')

In [19]:
# Mostrando archivos en el directorio de trabajo del entorno
!ls

X_test.npy  X_train.npy


In [20]:
# Imprimiendo el directorio de trabajo
!pwd

/home/wsuser/work


Descargando el archivo de salida Y_train y Y_test

In [21]:
# Descargando Y_train2.npy
m.download_url('https://mega.nz/file/tdohXarb#SRrt31GIT7H6gioCVm2KYF-sH6a8PHK8Z7Tr76oZb4I')

PosixPath('Y_train2.npy')

In [22]:
!ls

X_test.npy  X_train.npy  Y_train2.npy


In [23]:
# Descargando Y_test2.npy
m.download_url('https://mega.nz/file/Vc5TBSbY#D8pbUvpA3l2V0PJxCu7Meg8WJ3wPpT6X2jEoDsleCWw')

PosixPath('Y_test2.npy')

In [24]:
!ls

X_test.npy  X_train.npy  Y_test2.npy  Y_train2.npy


Leyendo lo archivos con numpy

In [25]:
import numpy as np

In [26]:
X_train = np.load('X_train.npy')
X_train.shape

(3337, 17)

In [27]:
X_test = np.load('X_test.npy')
X_test.shape

(832, 17)

In [28]:
Y_train = np.load('Y_train2.npy')
Y_train.shape

(3337,)

In [29]:
Y_test = np.load('Y_test2.npy')
Y_test.shape

(832,)

### Descargando el modelo

In [30]:
# # Descargando archivo con pesos entrenados rnet_weights.h5
m.download_url('https://mega.nz/file/8VhSBJIY#MuM4xdpkT64Jau9M52sk35pTo8BJIlNV9D53_UEK1Cc')

PosixPath('rnet_weights.h5')

In [31]:
# Descargando archivo del modelo completo PCA_rnet_FULL.h5
# m.download_url('https://mega.nz/file/Jdw0xRAb#niBMdBeTC0ZcPWhQz-pavMPrwLXru9tt6aVBOvO2u6s')

In [32]:
!ls

rnet_weights.h5  X_test.npy  X_train.npy  Y_test2.npy  Y_train2.npy


## PCA

Creando el modelo que reduce dimensionalmente X_test a partir de los datos X_train.

In [33]:
# Importando el método PCA
from sklearn.decomposition import PCA

In [34]:
# Instanciando PCa en un objeto que transforme los datos a 3 dimensiones
pca = PCA(n_components=3)

In [35]:
# Para X_train
X_train_pca = pca.fit(X_train).transform(X_train)

In [36]:
# Para X_test
# Nótese que se utiliza X_train para ajustar el pca
X_test_pca = pca.fit(X_train).transform(X_test)

In [37]:
# Comprobación con un ejemplo cualesquiera

#Leyendo ejemplo
ejemplo_test = X_test[5,:]

#Expandiendo una dimension
ejemplo_test = np.expand_dims(ejemplo_test,axis=0)

#Transormando el ejemplo
prueba1 = pca.fit(X_train).transform(ejemplo_test)
print(prueba1)
print(X_test_pca[5,:])

[[-18.52513566  21.87689658  -5.58046172]]
[-18.52513566  21.87689658  -5.58046172]


In [38]:
# Función para retornar vector de carcterísticas después de aplicar PCA
def PCA_transform(vector):
    
    #Expandiendo una dimension
    vector = np.expand_dims(vector,axis=0)
    
    #Transformando el ejemplo
    vector_pca = pca.fit(X_train).transform(vector)
    
    return vector_pca

In [42]:
#Prueba 2
ejemplo_test2 = X_test[515,:]
#Transformando el ejemplo
prueba2 = PCA_transform(ejemplo_test2)
print(prueba2)
print(X_test_pca[515,:])

[[-22.83520649   5.3994245    3.07702366]]
[-22.83520649   5.3994245    3.07702366]


## Configuración del modelo para su despliegue

Para desplegar el modelo predictivo de tensorflow en la nube es necesario comprimir el archivo h5 a un archivo .tgz

Se define el modelo

In [43]:
# # importando bibliotecas auxiliares
import tensorflow as tf
# from tensorflow.keras.applications import *
from tensorflow.keras.layers import *

Definiendo la arquitectura del modelo.

In [44]:
dropout = 0.2

In [45]:
# Creando el modelo de la red neuronal de tres capasocultas con 300 unidades por capa
rnet = tf.keras.Sequential()
rnet.add(Dense(300, activation="relu", input_shape=(X_train_pca.shape[1],)))
rnet.add(Dropout(dropout))
rnet.add(Dense(300, activation="relu"))
rnet.add(Dropout(dropout))
rnet.add(Dense(300, activation="relu"))
rnet.add(Dropout(dropout))
rnet.add(Dense(1, activation='sigmoid'))

Cargando los pesos previamente entrenados en Kaggle para el modelo rnet.

In [46]:
rnet.load_weights('rnet_weights.h5')

Guardando modelo nuevamente con los pesos entrenados

In [47]:
rnet.save('modelo.h5')

In [48]:
!ls

modelo.h5  rnet_weights.h5  X_test.npy	X_train.npy  Y_test2.npy  Y_train2.npy


Comprimiendo el modelo utilizando el comando *tar*

In [49]:
!tar -zcvf modelo.tar.gz modelo.h5

modelo.h5


In [50]:
!ls

modelo.h5      rnet_weights.h5	X_train.npy  Y_train2.npy
modelo.tar.gz  X_test.npy	Y_test2.npy


In [51]:
# Guardando la dirección del model
model_path = '/home/wsuser/work/modelo.tar.gz'

## Modelo de Tensorflow desplegado en la nube

Aquí se describen los pasos para almaenar el modelo EfficientNet en el repositorio de Watson Machine Learningn utilizando el SDK de IBM Watson Machine Learning.

### Publicando modelo

Definiendo nombre del modelo, nombre del autor y el software específico.

In [52]:
tf.__version__

'2.1.0'

In [53]:
sofware_spec_uid = client.software_specifications.get_id_by_name("tensorflow_2.1-py3.7")

In [54]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'Red Neuronal PCA',
            client.repository.ModelMetaNames.TYPE: 'tensorflow_2.1',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = client.repository.store_model(
    model=model_path,
    meta_props=metadata)

### Obteniendo detalles del modelo

In [55]:
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "software_spec": {
      "id": "c4032338-2a40-500a-beef-b01ab2667e27",
      "name": "tensorflow_2.1-py3.7"
    },
    "type": "tensorflow_2.1"
  },
  "metadata": {
    "created_at": "2020-12-04T11:31:44.178Z",
    "id": "dac8c7ec-782e-4fd4-b73d-ae93da0ae15c",
    "modified_at": "2020-12-04T11:31:46.780Z",
    "name": "Red Neuronal PCA",
    "owner": "IBMid-550009WDCT",
    "space_id": "27c26218-aade-455c-8e7a-6382b3abf19c"
  },
  "system": {
    "warnings": []
  }
}


In [56]:
models_details = client.repository.list_models()

------------------------------------  ----------------  ------------------------  --------------
ID                                    NAME              CREATED                   TYPE
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  Red Neuronal PCA  2020-12-04T11:31:44.002Z  tensorflow_2.1
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  EfficientNetB7    2020-12-04T02:17:10.002Z  tensorflow_2.1
------------------------------------  ----------------  ------------------------  --------------


## Desplegando y almacenando en una nube

Se almacena el modelo para poder obtener un score de forma online uutilizando solicitudes tipo con arreglos tipo json con ayuda de IBM Watson Machine Learning SDK.

### Creando modelo desplegado

In [57]:
from tensorflow import keras

#### Creando despliegue online para el modelo publicado.

In [58]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of external Tensorflow model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' started

#######################################################################################


initializing..
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
------------------------------------------------------------------------------------------------




In [59]:
deployment_uid = client.deployments.get_uid(created_deployment)

Ahora podemos imprimir el "online scoring endpoint"

In [60]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/ml/v4/deployments/XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX/predictions


You can also list existing deployments.

In [61]:
client.deployments.list()

------------------------------------  ---------------------------------------  -----  ------------------------
GUID                                  NAME                                     STATE  CREATED
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  Deployment of external Tensorflow model  ready  2020-12-04T11:32:19.864Z
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  Deployment of external Tensorflow model  ready  2020-12-04T02:17:40.026Z
------------------------------------  ---------------------------------------  -----  ------------------------


### Obteniendo detalles del "despliegue"

In [62]:
client.deployments.get_details(deployment_uid)

{'entity': {'asset': {'id': 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'},
  'custom': {},
  'deployed_asset_type': 'model',
  'hardware_spec': {'id': 'Not_Applicable', 'name': 'XS', 'num_nodes': 1},
  'name': 'Deployment of external Tensorflow model',
  'online': {},
  'space_id': '27c26218-aade-455c-8e7a-6382b3abf19c',
  'status': {'online_url': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX/predictions'},
   'state': 'ready'}},
 'metadata': {'created_at': '2020-12-04T11:32:19.864Z',
  'id': 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX',
  'modified_at': '2020-12-04T11:32:19.864Z',
  'name': 'Deployment of external Tensorflow model',
  'owner': 'IBMid-550009WDCT',
  'space_id': 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'}}

### Score

Ahora vamos a realizar una prueba de acceso.

In [63]:
score_0 = PCA_transform(X_test[515,:]).tolist()

In [64]:
scoring_payload = {"input_data": [{"values": [score_0[0]]}]}

Usando el método ``client.deployments.score()``para probar el modelo en la nube.

In [65]:
predictions = client.deployments.score(deployment_uid, scoring_payload)

Let's print the result of predictions.

In [66]:
import json
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "id": "dense_3",
      "fields": [
        "prediction",
        "prediction_classes",
        "probability"
      ],
      "values": [
        [
          [
            0.017080986872315407
          ],
          [
            0
          ],
          [
            0.017080986872315407
          ]
        ]
      ]
    }
  ]
}


Imprimimos la probabilidad de cáncer.

In [67]:
# Modelo Online
predictions['predictions'][0]['values'][0][0][0]

0.017080986872315407

In [68]:
# Modelo "local"
rnet.predict(score_0)[0][0]

0.01708098